# HRNet

In [ ]:
!python demo/topdown_img_demo.py \
    /data/Datasets/Fish/FIB/FIB/inference/IMG_0129-Editar.jpg \
    configs/animal_2d_keypoint/ak/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256.py \
    checkpoints/P3_fishes_hrnet_w32.pth \
    --out-img-root vis_results/ \
    --draw-heatmap

usage: topdown_demo_with_mmdet.py [-h] [--input INPUT] [--show]
                                  [--output-root OUTPUT_ROOT]
                                  [--save-predictions] [--device DEVICE]
                                  [--det-cat-id DET_CAT_ID]
                                  [--bbox-thr BBOX_THR] [--nms-thr NMS_THR]
                                  [--kpt-thr KPT_THR] [--draw-heatmap]
                                  [--show-kpt-idx]
                                  [--skeleton-style {mmpose,openpose}]
                                  [--radius RADIUS] [--thickness THICKNESS]
                                  [--show-interval SHOW_INTERVAL]
                                  [--alpha ALPHA] [--draw-bbox]
                                  det_config det_checkpoint pose_config
                                  pose_checkpoint
topdown_demo_with_mmdet.py: error: unrecognized arguments: --out-img-root


In [1]:
!python demo/image_demo_hrnet.py \
  /data/Datasets/Fish/2022/ \
  configs/animal_2d_keypoint/topdown_heatmap/ak/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256.py \
  checkpoints/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256-76c3999f_20230519.pth \
  --out-file vis_results/fish_skeleton.jpg \
  --draw-heatmap \
  --show-kpt-idx \
  --device cuda


Loads checkpoint by local backend from path: checkpoints/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256-76c3999f_20230519.pth


DEBUG: Loaded model.dataset_meta keys = dict_keys(['dataset_name', 'num_keypoints', 'keypoint_id2name', 'keypoint_name2id', 'upper_body_ids', 'lower_body_ids', 'flip_indices', 'flip_pairs', 'keypoint_colors', 'num_skeleton_links', 'skeleton_links', 'skeleton_link_colors', 'dataset_keypoint_weights', 'sigmas', 'CLASSES']) 


Traceback (most recent call last):
  File "/home/arumota_pupils/Josue/1PECES/MMPose/mmpose/demo/image_demo_hrnet.py", line 78, in <module>
    main()
  File "/home/arumota_pupils/Josue/1PECES/MMPose/mmpose/demo/image_demo_hrnet.py", line 53, in main
    batch_results = inference_topdown(model, args.img)
  File "/home/arumota_pupils/Josue/1PECES/MMPose/mmpose/mmpose/apis/inference.py", line 163, in inference_topdown
    w, h = Image.open(img).size
  File "/opt/conda/lib/python3.10/site-packages/PIL/Image.py", line 3247, in open
   

In [2]:
import os
import glob
import subprocess
import json

# Paths
input_folder = '/data/Datasets/Fish/2022/images/val_fish/'
output_folder = '/data/Datasets/Fish/2022/output_hrnet/'
config_path = 'configs/animal_2d_keypoint/topdown_heatmap/ak/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256_train.py'
checkpoint_path = 'work_dirs/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256_train/epoch_600.pth'
merged_json_path = os.path.join(output_folder, 'all_keypoints_predicted.json')

os.makedirs(output_folder, exist_ok=True)

# Get image list
image_files = sorted(glob.glob(os.path.join(input_folder, '*.[jp][pn]g')))

# Acumulador para resultados
all_predictions = []

# Loop por imagen
for img_path in image_files:
    img_name = os.path.basename(img_path)
    out_base = os.path.join(output_folder, img_name)
    out_json = out_base.replace('.jpg', '_keypoints.json').replace('.png', '_keypoints.json')

    cmd = [
        "python", "demo/image_demo.py",
        img_path,
        config_path,
        checkpoint_path,
        "--out-file", out_base,
        "--draw-heatmap",
        "--show-kpt-idx"
    ]

    print(f"🔁 Procesando {img_name}...")
    subprocess.run(cmd)

    # Cargar JSON individual generado por el demo
    if os.path.exists(out_json):
        with open(out_json, 'r') as f:
            pred = json.load(f)
            all_predictions.append(pred)
    else:
        print(f"⚠️ No se encontró JSON para {img_name}")

# Guardar todos en un único archivo
with open(merged_json_path, 'w') as f:
    json.dump(all_predictions, f, indent=4)

print(f"\n✅ JSON consolidado guardado en: {merged_json_path}")


🔁 Procesando DSC00006_27022023165306.jpg...
Loads checkpoint by local backend from path: work_dirs/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256_train/epoch_600.pth


✅ DEBUG: Patched model.dataset_meta keys = dict_keys(['dataset_name', 'num_keypoints', 'keypoint_names', 'keypoint_id2name', 'keypoint_name2id', 'skeleton', 'keypoint_colors', 'skeleton_links_color', 'flip_pairs', 'flip_indices', 'upper_body_ids', 'lower_body_ids', 'dataset_keypoint_weights', 'sigmas', 'CLASSES']) 


📍 Coordenadas predichas:
 - snout: x=164.4, y=885.5, score=0.919

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/2022/output_hrnet/DSC00006_27022023165306_keypoints.json
 - caudalmid: x=1397.8, y=621.2, score=0.506

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/2022/output_hrnet/DSC00006_27022023165306_keypoints.json
 - dorsal: x=996.8, y=611.8, score=0.371

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/2022/output_hrnet/DSC00006_27022023165306_keypoints.json
 - pelvic: x=1066.5

In [ ]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Rutas
pred_file = Path("/data/Datasets/Fish/2022/output_hrnet/all_keypoints_predicted.json")
gt_file = Path("/data/Datasets/Fish/2022/fish8kpt_val.json")

# Cargar archivos
with pred_file.open("r") as f:
    pred_data = json.load(f)

with gt_file.open("r") as f:
    gt_data = json.load(f)

# Crear índice de anotaciones por imagen
image_id_to_name = {img['id']: img['file_name'] for img in gt_data['images']}
annotations_by_name = {
    image_id_to_name[ann['image_id']]: np.array(ann['keypoints']).reshape(-1, 3)[:, :2]
    for ann in gt_data['annotations']
}

# Comparación punto a punto
rows = []
gt_all = []
pred_all = []

for item in pred_data:
    name = item["image"]
    pred_kpts = {kp["name"]: (kp["x"], kp["y"]) for kp in item["keypoints"]}
    gt_kpts = annotations_by_name.get(name)

    if gt_kpts is None:
        print(f"⚠️ Imagen {name} no encontrada en GT.")
        continue

    for idx, (gt_xy, (kpt_name, pred_xy)) in enumerate(zip(gt_kpts, pred_kpts.items())):
        error = np.linalg.norm(np.array(pred_xy) - np.array(gt_xy))
        rows.append({
            "image": name,
            "keypoint": kpt_name,
            "gt_x": gt_xy[0],
            "gt_y": gt_xy[1],
            "pred_x": pred_xy[0],
            "pred_y": pred_xy[1],
            "euclidean_error": error
        })
        gt_all.extend(gt_xy)
        pred_all.extend(pred_xy)

df = pd.DataFrame(rows)

# --- 1. Métricas globales por keypoint ---
trait_metrics = []
for kpt in df["keypoint"].unique():
    sub = df[df["keypoint"] == kpt]
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    trait_metrics.append({
        "keypoint": kpt,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy)
    })
df_trait = pd.DataFrame(trait_metrics)
df_trait.to_csv("/data/Datasets/Fish/2022/hrnet/metrics_per_keypoint.csv", index=False)

# --- 2. Métricas por imagen ---
image_metrics = []
for img in df["image"].unique():
    sub = df[df["image"] == img]
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    image_metrics.append({
        "image": img,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy)
    })
df_image = pd.DataFrame(image_metrics)
df_image.to_csv("/data/Datasets/Fish/2022/hrnet/metrics_per_image.csv", index=False)

# --- 3. Métricas por keypoint e imagen ---
trait_image_metrics = []
for (img, kpt), sub in df.groupby(["image", "keypoint"]):
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    trait_image_metrics.append({
        "image": img,
        "keypoint": kpt,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy) if len(gt_xy) > 1 else None
    })
df_trait_image = pd.DataFrame(trait_image_metrics)
df_trait_image.to_csv("/data/Datasets/Fish/2022/hrnet/metrics_per_keypoint_image.csv", index=False)

print("✅ ¡Listo! Se guardaron los tres archivos CSV:")
print("- metrics_per_keypoint.csv")
print("- metrics_per_image.csv")
print("- metrics_per_keypoint_image.csv")

⚠️ Imagen DSC00006_27022023165306.jpg no encontrada en GT.
⚠️ Imagen DSC02823_08032023153633.jpg no encontrada en GT.
⚠️ Imagen DSC03012_10032023194229.jpg no encontrada en GT.
⚠️ Imagen DSC03032_10032023200032.jpg no encontrada en GT.
✅ ¡Listo! Se guardaron los tres archivos CSV:
- metrics_per_keypoint.csv
- metrics_per_image.csv
- metrics_per_keypoint_image.csv


In [4]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

# Rutas de entrada
pred_file = Path("/data/Datasets/Fish/2022/output_hrnet/all_keypoints_predicted.json")
gt_file = Path("/data/Datasets/Fish/2022/fish8kpt_val.json")

# Cargar JSONs
with pred_file.open("r") as f:
    pred_data = json.load(f)

with gt_file.open("r") as f:
    gt_data = json.load(f)

# Indexar ground-truth por nombre de imagen
image_id_to_name = {img["id"]: img["file_name"] for img in gt_data["images"]}
annotations_by_name = {
    image_id_to_name[ann["image_id"]]: np.array(ann["keypoints"]).reshape(-1, 3)
    for ann in gt_data["annotations"]
}

# Armar dataset completo
rows = []
for pred_item in pred_data:
    name = pred_item["image"]
    gt_kpts = annotations_by_name.get(name)
    if gt_kpts is None:
        print(f"⚠️ Imagen {name} no está en GT.")
        continue

    for idx, pred_kpt in enumerate(pred_item["keypoints"]):
        gt_x, gt_y = gt_kpts[idx][:2]
        pred_x, pred_y = pred_kpt["x"], pred_kpt["y"]
        score = pred_kpt["score"]
        error = np.linalg.norm([gt_x - pred_x, gt_y - pred_y])

        rows.append({
            "image": name,
            "keypoint": pred_kpt["name"],
            "gt_x": gt_x,
            "gt_y": gt_y,
            "pred_x": pred_x,
            "pred_y": pred_y,
            "score": score,
            "euclidean_error": error
        })

# Guardar CSV
df_all = pd.DataFrame(rows)
df_all.to_csv("/data/Datasets/Fish/2022/hrnet/keypoints_detailed_comparison.csv", index=False)
print("✅ Archivo generado: keypoints_detailed_comparison.csv")


⚠️ Imagen DSC00006_27022023165306.jpg no está en GT.
⚠️ Imagen DSC02823_08032023153633.jpg no está en GT.
⚠️ Imagen DSC03012_10032023194229.jpg no está en GT.
⚠️ Imagen DSC03032_10032023200032.jpg no está en GT.
✅ Archivo generado: keypoints_detailed_comparison.csv


In [ ]:
Objetivo General
Desarrollar un programa para la prevención, diagnóstico temprano, monitorización y pronóstico del accidente cerebrovascular, utilizando una plataforma de salud digital que involucre el modelamiento de hallazgos radiológicos, la inclusión de variables clínicas y las escalas de estratificación de pacientes para la atención remota de comunidades con restricciones de acceso y dificultades socio-económicas.
Objetivos Específicos
Objetivo Específico 1 (OE1 - Educación): Desarrollar jornadas educativas para la sensibilización en comunidades vulnerables sobre el accidente cerebrovascular, la inteligencia artificial y la salud digital como soporte de atención a profesionales, pobladores y redes de atención en salud que actúan en sitios remotos
Objetivo Específico 2  (OE2 - Tamizaje):  Realizar un esquema de  tamizaje y caracterización poblacional de las comunidades locales en cuanto accidente cerebro vascular recopilando variables clínicas, factores de riesgo  e índices epidemiológicos que permitan codificar protocolos de atención en zonas alejadas de centros primarios de atención.
Objetivo Específico 3 (OE3 - Diagnóstico): Desarrollar modelos multimodales de inteligencia artificial para soportar el diagnóstico y pronóstico del accidente cerebrovascular a través de la caracterización morfométrica de las lesiones, así como también que integren valoraciones clínicas que puedan prever estrategias de tratamiento y la evaluación de la recuperación post-tratamiento en pacientes con accidente cerebrovascular.
Objetivo Específico 4 (OE4 - Monitoreo): Ajustar modelos fundacionales de inteligencia artificial para brindar soporte a profesionales en el monitoreo y detección de accidentes cerebrovasculares, así como también para integrar información clínica y variables imagenológicas disponibles para una estratificación personalizada de la afectación del paciente.
Objetivo Específico 5 (OE5 - Atención Integral): Desarrollar mecanismos de atención integral para la población caracterizada, apoyados en una infraestructura de telemedicina que facilite el acceso a servicios médicos especializados tanto para la prevención como para el seguimiento continuo en casos reportados.


## Training

In [ ]:
!python tools/train.py \
  /home/arumota_pupils/Josue/1PECES/MMPose/mmpose/configs/animal_2d_keypoint/topdown_heatmap/ak/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256_train.py 
  
!export PYTHONPATH=$(pwd) && python tools/train.py configs/animal_2d_keypoint/topdown_heatmap/ak/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256_train.py 

^C


# VitPose

## Training script

Must be copied on a terminal without the ! mark

In [ ]:
!cd /home/arumota_pupils/Josue/1PECES/MMPose/mmpose
    
!python tools/train.py \
    /home/arumota_pupils/Josue/1PECES/MMPose/mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain.py

05/06 09:59:52 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 819089920
    GPU 0: NVIDIA GeForce RTX 4090
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.1, V12.1.105
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.2.1
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-genc

## Inference over weights

Inference over a single image

In [9]:
# 1. La imagen a inferir (Ruta)
# 2. Configuración usada para entrenar (Ruta)
# 3. Ruta de los pesos

!python demo/image_demo.py \
    "/data/Datasets/Fish/2022/images/val_fish/DSC03012_10032023194229.jpg" \
    "configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain.py" \
    "/data/Josue/weights/Fish/epoch_final_vit.pth" \
    --out-file "work_dirs/images/test_file.png" \
    --draw-heatmap \
    --show-kpt-idx

Loads checkpoint by local backend from path: /data/Josue/weights/Fish/epoch_final_vit.pth


✅ DEBUG: Patched model.dataset_meta keys = dict_keys(['dataset_name', 'num_keypoints', 'keypoint_names', 'keypoint_id2name', 'keypoint_name2id', 'skeleton', 'keypoint_colors', 'skeleton_links_color', 'flip_pairs', 'flip_indices', 'upper_body_ids', 'lower_body_ids', 'dataset_keypoint_weights', 'sigmas', 'CLASSES']) 


📍 Coordenadas predichas:
 - snout: x=1885.3, y=679.1, score=1.003

📝 Coordenadas guardadas en JSON: work_dirs/images/test_file_keypoints.json
 - caudalmid: x=368.1, y=690.9, score=1.032

📝 Coordenadas guardadas en JSON: work_dirs/images/test_file_keypoints.json
 - dorsal: x=1346.1, y=522.9, score=0.961

📝 Coordenadas guardadas en JSON: work_dirs/images/test_file_keypoints.json
 - pelvic: x=1273.7, y=768.0, score=0.847

📝 Coordenadas guardadas en JSON: work_dirs/images/test_file_keypoints.json
 - nape: x=1720.5, y=535.0, score=0.993

📝 Coordenadas guardadas en JSON: work_dirs/images/

Inference over a set of image definied over the input_folder path

In [1]:
import os
import glob

# Define paths
input_folder = '/data/Datasets/Fish/2022/images/val_fish/'
output_folder = '/data/Datasets/Fish/2022/output/'
config_path = 'configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain.py'
checkpoint_path = '/data/Josue/weights/Fish/epoch_final_vit.pth'

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get all image files (jpg/png/jpeg)
image_files = sorted(glob.glob(os.path.join(input_folder, '*.[jp][pn]g')))

# Loop through and run demo script
for img_path in image_files:
    img_name = os.path.basename(img_path)
    out_file = os.path.join(output_folder, img_name)

    !python demo/image_demo.py \
        "{img_path}" \
        "{config_path}" \
        "{checkpoint_path}" \
        --out-file "{out_file}" \
        --draw-heatmap \
        --show-kpt-idx

Loads checkpoint by local backend from path: /data/Josue/weights/Fish/epoch_final_vit.pth


✅ DEBUG: Patched model.dataset_meta keys = dict_keys(['dataset_name', 'num_keypoints', 'keypoint_names', 'keypoint_id2name', 'keypoint_name2id', 'skeleton', 'keypoint_colors', 'skeleton_links_color', 'flip_pairs', 'flip_indices', 'upper_body_ids', 'lower_body_ids', 'dataset_keypoint_weights', 'sigmas', 'CLASSES']) 


📍 Coordenadas predichas:
 - snout: x=135.4, y=900.6, score=1.026

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/2022/output/DSC00006_27022023165306.jpg
 - caudalmid: x=1851.7, y=572.7, score=0.992

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/2022/output/DSC00006_27022023165306.jpg
 - dorsal: x=570.5, y=656.9, score=0.975

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/2022/output/DSC00006_27022023165306.jpg
 - pelvic: x=627.7, y=839.1, score=0.954

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/2022/output/DSC00006_27022023165306.jpg
 - nape: x=177.5, y=8

In [3]:
import os
import glob
import subprocess
import json

# Paths
input_folder = '/data/Datasets/Fish/2022/images/val_fish/'
output_folder = '/data/Datasets/Fish/2022/output/'
config_path = 'configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain.py'
checkpoint_path = '/data/Josue/weights/Fish/epoch_final_vit.pth'
merged_json_path = os.path.join(output_folder, 'all_keypoints_predicted.json')

os.makedirs(output_folder, exist_ok=True)

# Get image list
image_files = sorted(glob.glob(os.path.join(input_folder, '*.[jp][pn]g')))

# Acumulador para resultados
all_predictions = []

# Loop por imagen
for img_path in image_files:
    img_name = os.path.basename(img_path)
    out_base = os.path.join(output_folder, img_name)
    out_json = out_base.replace('.jpg', '_keypoints.json').replace('.png', '_keypoints.json')

    cmd = [
        "python", "demo/image_demo.py",
        img_path,
        config_path,
        checkpoint_path,
        "--out-file", out_base,
        "--draw-heatmap",
        "--show-kpt-idx"
    ]

    print(f"🔁 Procesando {img_name}...")
    subprocess.run(cmd)

    # Cargar JSON individual generado por el demo
    if os.path.exists(out_json):
        with open(out_json, 'r') as f:
            pred = json.load(f)
            all_predictions.append(pred)
    else:
        print(f"⚠️ No se encontró JSON para {img_name}")

# Guardar todos en un único archivo
with open(merged_json_path, 'w') as f:
    json.dump(all_predictions, f, indent=4)

print(f"\n✅ JSON consolidado guardado en: {merged_json_path}")


🔁 Procesando DSC00006_27022023165306.jpg...
Loads checkpoint by local backend from path: /data/Josue/weights/Fish/epoch_final_vit.pth


✅ DEBUG: Patched model.dataset_meta keys = dict_keys(['dataset_name', 'num_keypoints', 'keypoint_names', 'keypoint_id2name', 'keypoint_name2id', 'skeleton', 'keypoint_colors', 'skeleton_links_color', 'flip_pairs', 'flip_indices', 'upper_body_ids', 'lower_body_ids', 'dataset_keypoint_weights', 'sigmas', 'CLASSES']) 


📍 Coordenadas predichas:
 - snout: x=135.4, y=900.6, score=1.026

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/2022/output/DSC00006_27022023165306_keypoints.json
 - caudalmid: x=1851.7, y=572.7, score=0.992

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/2022/output/DSC00006_27022023165306_keypoints.json
 - dorsal: x=570.5, y=656.9, score=0.975

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/2022/output/DSC00006_27022023165306_keypoints.json
 - pelvic: x=627.7, y=839.1, score=0.954

📝 Coordenadas guardadas en JSON: /dat

In [18]:
import json
import random
import numpy as np
import matplotlib.pyplot as plt
from mmengine import load
from PIL import Image
from pathlib import Path

# Cargar el archivo val.json
val_json_path = Path("/data/Datasets/Fish/2022/fish8kpt_val.json")
with open(val_json_path, "r") as f:
    val_data = json.load(f)

# Cargar las predicciones
results = load("/data/Josue/weights/Fish/epoch_600.pth")  # Asegúrate que este path es correcto o reemplázalo

# Mapear image_id a file_path
id_to_filename = {img["id"]: img["file_name"] for img in val_data["images"]}
image_dir = Path(val_json_path).parent  # /data/Datasets/Fish/2022

# Seleccionar una predicción aleatoria que esté en el val set
val_ids = set(id_to_filename.keys())
sample = next(r for r in results if r.img_id in val_ids)

# Obtener keypoints predichos
kpts = sample.pred_instances.keypoints  # (K, 3)
img_path = image_dir / id_to_filename[sample.img_id]
img = Image.open(img_path)

# Mostrar imagen con keypoints
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.axis("off")

for x, y, v in kpts:
    if v > 0:
        plt.scatter(x, y, c='red', s=40)
        plt.text(x + 2, y + 2, f"{int(x)},{int(y)}", fontsize=8, color='yellow')

plt.title(f"Predicted Keypoints for Image ID: {sample.img_id}")
plt.show()


TypeError: Unsupported format: pth

In [7]:
!python demo/image_demo.py \
  /data/Datasets/Fish/FIB/FIB/inference/IMG_0181-Editar.jpg \
  configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etest.py \
  work_dirs/td-hm_ViTPose-fish9_8xb32-100etrain/epoch_600.pth \
  --out-file vis_results/vitpose_600epoch/fish_infer_0181.jpg \
  --draw-heatmap \
  --show-kpt-idx

Loads checkpoint by local backend from path: work_dirs/td-hm_ViTPose-fish9_8xb32-100etrain/epoch_600.pth


DEBUG: Patched model.dataset_meta keys = dict_keys(['dataset_name', 'num_keypoints', 'keypoint_id2name', 'keypoint_name2id', 'keypoint_names', 'skeleton', 'keypoint_colors', 'skeleton_links_color', 'flip_pairs', 'flip_indices', 'upper_body_ids', 'lower_body_ids', 'dataset_keypoint_weights', 'sigmas', 'CLASSES']) 


04/09 05:10:06 - mmengine - INFO - The output image has been saved at vis_results/vitpose_600epoch/fish_infer_0181.jpg


# [Métricas] Comparación de puntos

In [4]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Rutas
pred_file = Path("/data/Datasets/Fish/2022/output/all_keypoints_predicted.json")
gt_file = Path("/data/Datasets/Fish/2022/fish8kpt_val.json")

# Cargar archivos
with pred_file.open("r") as f:
    pred_data = json.load(f)

with gt_file.open("r") as f:
    gt_data = json.load(f)

# Crear índice de anotaciones por imagen
image_id_to_name = {img['id']: img['file_name'] for img in gt_data['images']}
annotations_by_name = {
    image_id_to_name[ann['image_id']]: np.array(ann['keypoints']).reshape(-1, 3)[:, :2]
    for ann in gt_data['annotations']
}

# Comparación punto a punto
rows = []
gt_all = []
pred_all = []

for item in pred_data:
    name = item["image"]
    pred_kpts = {kp["name"]: (kp["x"], kp["y"]) for kp in item["keypoints"]}
    gt_kpts = annotations_by_name.get(name)

    if gt_kpts is None:
        print(f"⚠️ Imagen {name} no encontrada en GT.")
        continue

    for idx, (gt_xy, (kpt_name, pred_xy)) in enumerate(zip(gt_kpts, pred_kpts.items())):
        error = np.linalg.norm(np.array(pred_xy) - np.array(gt_xy))
        rows.append({
            "image": name,
            "keypoint": kpt_name,
            "gt_x": gt_xy[0],
            "gt_y": gt_xy[1],
            "pred_x": pred_xy[0],
            "pred_y": pred_xy[1],
            "euclidean_error": error
        })
        gt_all.extend(gt_xy)
        pred_all.extend(pred_xy)

df = pd.DataFrame(rows)

# --- 1. Métricas globales por keypoint ---
trait_metrics = []
for kpt in df["keypoint"].unique():
    sub = df[df["keypoint"] == kpt]
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    trait_metrics.append({
        "keypoint": kpt,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy)
    })
df_trait = pd.DataFrame(trait_metrics)
df_trait.to_csv("/data/Datasets/Fish/2022/metrics_per_keypoint.csv", index=False)

# --- 2. Métricas por imagen ---
image_metrics = []
for img in df["image"].unique():
    sub = df[df["image"] == img]
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    image_metrics.append({
        "image": img,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy)
    })
df_image = pd.DataFrame(image_metrics)
df_image.to_csv("/data/Datasets/Fish/2022/metrics_per_image.csv", index=False)

# --- 3. Métricas por keypoint e imagen ---
trait_image_metrics = []
for (img, kpt), sub in df.groupby(["image", "keypoint"]):
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    trait_image_metrics.append({
        "image": img,
        "keypoint": kpt,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy) if len(gt_xy) > 1 else None
    })
df_trait_image = pd.DataFrame(trait_image_metrics)
df_trait_image.to_csv("/data/Datasets/Fish/2022/metrics_per_keypoint_image.csv", index=False)

print("✅ ¡Listo! Se guardaron los tres archivos CSV:")
print("- metrics_per_keypoint.csv")
print("- metrics_per_image.csv")
print("- metrics_per_keypoint_image.csv")

⚠️ Imagen DSC00006_27022023165306.jpg no encontrada en GT.
⚠️ Imagen DSC02823_08032023153633.jpg no encontrada en GT.
⚠️ Imagen DSC03012_10032023194229.jpg no encontrada en GT.
⚠️ Imagen DSC03032_10032023200032.jpg no encontrada en GT.


✅ ¡Listo! Se guardaron los tres archivos CSV:
- metrics_per_keypoint.csv
- metrics_per_image.csv
- metrics_per_keypoint_image.csv


⚠️ Imagen DSC00006_27022023165306.jpg no está en el set de validación.
⚠️ Imagen DSC02823_08032023153633.jpg no está en el set de validación.
⚠️ Imagen DSC03012_10032023194229.jpg no está en el set de validación.
⚠️ Imagen DSC03032_10032023200032.jpg no está en el set de validación.
✅ Comparación por punto guardada en 'keypoint_comparison.csv'


TypeError: got an unexpected keyword argument 'squared'

# Formatting code

## Splitting into train/test with a 10 ratio

In [6]:
import json
import random
from pathlib import Path

# Load full annotation file (Compressed body as example)
input_path = Path("/data/Datasets/Fish/2022/fisk_8kpts.json")
with input_path.open("r") as f:
    data = json.load(f)

# Shuffle and split
random.seed(42)
image_ids = list({img["id"] for img in data["images"]})
random.shuffle(image_ids)
split_index = int(0.9 * len(image_ids))
train_ids = set(image_ids[:split_index])
val_ids = set(image_ids[split_index:])

# Build separate image/annotation lists
def split_coco(data, selected_ids):
    return {
        "info": data.get("info", {}),
        "licenses": data.get("licenses", []),
        "images": [img for img in data["images"] if img["id"] in selected_ids],
        "annotations": [ann for ann in data["annotations"] if ann["image_id"] in selected_ids],
        "categories": data["categories"]
    }

train_data = split_coco(data, train_ids)
val_data = split_coco(data, val_ids)

# Save split files
train_path = "/data/Datasets/Fish/2022/fish8kpt_train.json"
val_path = "/data/Datasets/Fish/2022/fish8kpt_val.json"
with open(train_path, "w") as f:
    json.dump(train_data, f, indent=2)
with open(val_path, "w") as f:
    json.dump(val_data, f, indent=2)

train_path, val_path

('/data/Datasets/Fish/2022/fish8kpt_train.json',
 '/data/Datasets/Fish/2022/fish8kpt_val.json')